In [10]:
from datasets import load_dataset
import pandas as pd
import re

In [2]:
# Load English training split
train_dataset = load_dataset("ju-resplande/askD", split="train_en")

print(train_dataset[0])

Generating train_pt split: 0 examples [00:00, ? examples/s]

Generating train_en split: 0 examples [00:00, ? examples/s]

Generating validation_pt split: 0 examples [00:00, ? examples/s]

Generating validation_en split: 0 examples [00:00, ? examples/s]

Generating test_pt split: 0 examples [00:00, ? examples/s]

Generating test_en split: 0 examples [00:00, ? examples/s]

Generating external_pt split: 0 examples [00:00, ? examples/s]

Generating external_en split: 0 examples [00:00, ? examples/s]

{'q_id': '3yzn3p', 'title': 'Chest discomfort question', 'selftext': 'On and off for the last year or so, I\'ve been having occasional discomfort in my chest, particularly on the left side. The first time it happened, I was understandably worried and went to the emergency room. After some X-rays and an hour on an EKG, they sent me home saying they couldn\'t find evidence of anything wrong. Since then, it\'s happened several more times, occasionally accompanied by a feeling of shortness of breath. Whenever it happens, I check my pulse and nothing seems wrong (occasionally the pulse line looks a bit odd on my phone, but that could easily be an issue caused by my hand moving or something) and I never actually have any trouble drawing in breath.  Over the last few days, this has been recurring periodically, along with an occasional feeling of tightness in my throat and one or two bouts of transient weakness.\n\nAs background, I had two spontaneous pneumothoraxes in my left lung a little ov

In [4]:
train_dataset.to_parquet("../data/raw/askdocs/askdocs_train_en.parquet")


Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

45505564

In [28]:


# Load Parquet
df = pd.read_parquet("../data/raw/askdocs/askdocs_train_en.parquet")


In [29]:
len(df)

24256

In [30]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,3yzn3p,Chest discomfort question,"On and off for the last year or so, I've been ...",,AskDocs,"{'a_id': ['cyi6ywp'], 'text': ['This could be ...",[],[],[]
1,3z0cuf,Firecracker exploded close to ear. Ears still ...,So i was messing around with firecrackers and ...,,AskDocs,"{'a_id': ['cyi9mz0', 'cyi8vl0'], 'text': ['I a...",[],[],[]
2,3z0mlc,If a Schizophrenic patient is positive-symptom...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyic4j5', 'cyij9yl'], 'text': ['I d...",[],[],[]
3,3z0o23,Why all antipsychotics are more or less sedati...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyihyla'], 'text': ['This is believ...",[],[],[http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
4,3z25e8,"I have a very embarrassing problem, if anyone ...","Hey, I'm a 19 year old male and I've had this ...",,AskDocs,"{'a_id': ['cyit9ci'], 'text': ['Pin worms? Goo...",[],[],[]


In [31]:
# --- Drop [deleted]/[removed]/empty ---
BAD = {"[deleted]", "[removed]"}
df = df[~df["selftext"].astype(str).str.strip().isin(BAD)]
df = df[~df["title"].astype(str).str.strip().isin(BAD)]
df = df[df["selftext"].astype(str).str.strip() != ""]

# --- Cleaning function ---
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.strip()
    # strip markdown markers
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)   # **bold**
    text = re.sub(r"\*(.*?)\*", r"\1", text)       # *italics*
    text = re.sub(r"`(.*?)`", r"\1", text)         # `code`
    text = re.sub(r"\n{2,}", "\n", text)           # collapse newlines
    return text



In [32]:
len(df)

19560

In [33]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,3yzn3p,Chest discomfort question,"On and off for the last year or so, I've been ...",,AskDocs,"{'a_id': ['cyi6ywp'], 'text': ['This could be ...",[],[],[]
1,3z0cuf,Firecracker exploded close to ear. Ears still ...,So i was messing around with firecrackers and ...,,AskDocs,"{'a_id': ['cyi9mz0', 'cyi8vl0'], 'text': ['I a...",[],[],[]
2,3z0mlc,If a Schizophrenic patient is positive-symptom...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyic4j5', 'cyij9yl'], 'text': ['I d...",[],[],[]
3,3z0o23,Why all antipsychotics are more or less sedati...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyihyla'], 'text': ['This is believ...",[],[],[http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
4,3z25e8,"I have a very embarrassing problem, if anyone ...","Hey, I'm a 19 year old male and I've had this ...",,AskDocs,"{'a_id': ['cyit9ci'], 'text': ['Pin worms? Goo...",[],[],[]


In [34]:
# --- Keep clean columns ---
df["title_clean"] = df["title"].map(clean_text)
df["selftext_clean"] = df["selftext"].map(clean_text)

In [35]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,title_clean,selftext_clean
0,3yzn3p,Chest discomfort question,"On and off for the last year or so, I've been ...",,AskDocs,"{'a_id': ['cyi6ywp'], 'text': ['This could be ...",[],[],[],Chest discomfort question,"On and off for the last year or so, I've been ..."
1,3z0cuf,Firecracker exploded close to ear. Ears still ...,So i was messing around with firecrackers and ...,,AskDocs,"{'a_id': ['cyi9mz0', 'cyi8vl0'], 'text': ['I a...",[],[],[],Firecracker exploded close to ear. Ears still ...,So i was messing around with firecrackers and ...
2,3z0mlc,If a Schizophrenic patient is positive-symptom...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyic4j5', 'cyij9yl'], 'text': ['I d...",[],[],[],If a Schizophrenic patient is positive-symptom...,Dear /r/AskDocs: I don't know such medicine-re...
3,3z0o23,Why all antipsychotics are more or less sedati...,Dear /r/AskDocs: I don't know such medicine-re...,,AskDocs,"{'a_id': ['cyihyla'], 'text': ['This is believ...",[],[],[http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,Why all antipsychotics are more or less sedati...,Dear /r/AskDocs: I don't know such medicine-re...
4,3z25e8,"I have a very embarrassing problem, if anyone ...","Hey, I'm a 19 year old male and I've had this ...",,AskDocs,"{'a_id': ['cyit9ci'], 'text': ['Pin worms? Goo...",[],[],[],"I have a very embarrassing problem, if anyone ...","Hey, I'm a 19 year old male and I've had this ..."


In [ ]:
# --- Sample 5 for practice ---
sampled_train_df = df.sample(n=5, random_state=42).reset_index(drop=True)

In [37]:
sampled_train_df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,title_clean,selftext_clean
0,35p2z2,Should I get my liver checked?,"24, F, 5""4-ish, 55kg, White, last month or two...",,AskDocs,"{'a_id': ['cr6o7ce', 'cr6g59y'], 'text': ['com...",[],[],[http://www.drinksmarter.org/handy-tools/unit-...,Should I get my liver checked?,"24, F, 5""4-ish, 55kg, White, last month or two..."
1,8takb2,A strange migrane...starting to ruin my summer...,Ok so I’m 15 a guy roughly 5”9 and 130 pounds ...,,AskDocs,"{'a_id': ['e16fbn5', 'e16sels'], 'text': ['Sou...",[],[],[],A strange migrane...starting to ruin my summer...,Ok so I’m 15 a guy roughly 5”9 and 130 pounds ...
2,2gdooo,Rigor with antibiotics? Should I go to the hos...,I'm a sixteen year old female who has been pre...,,AskDocs,"{'a_id': ['cki7ezg'], 'text': ['This could be ...",[],[],[],Rigor with antibiotics? Should I go to the hos...,I'm a sixteen year old female who has been pre...
3,38xo1c,"Lost ~1.5L of blood, feeling very dizzy and we...","I'm a 19 year old male, 5'9"", and I had a sept...",,AskDocs,"{'a_id': ['cryoumi', 'crytvxa'], 'text': ['How...",[],[],[],"Lost ~1.5L of blood, feeling very dizzy and we...","I'm a 19 year old male, 5'9"", and I had a sept..."
4,2z1869,"My rash has stumped an NP, a PA, and three MDs","* 30\n* female\n* 5' 8""\n* 135 pre-pregnancy, ...",,AskDocs,"{'a_id': ['cpesedm', 'cpetqe4', 'cpf5d7o', 'cp...",[],[http://imgur.com/a/M5ZRj],[],"My rash has stumped an NP, a PA, and three MDs","* 30\n* female\n* 5' 8""\n* 135 pre-pregnancy, ..."


In [38]:
sampled_train_df

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,title_clean,selftext_clean
0,35p2z2,Should I get my liver checked?,"24, F, 5""4-ish, 55kg, White, last month or two...",,AskDocs,"{'a_id': ['cr6o7ce', 'cr6g59y'], 'text': ['com...",[],[],[http://www.drinksmarter.org/handy-tools/unit-...,Should I get my liver checked?,"24, F, 5""4-ish, 55kg, White, last month or two..."
1,8takb2,A strange migrane...starting to ruin my summer...,Ok so I’m 15 a guy roughly 5”9 and 130 pounds ...,,AskDocs,"{'a_id': ['e16fbn5', 'e16sels'], 'text': ['Sou...",[],[],[],A strange migrane...starting to ruin my summer...,Ok so I’m 15 a guy roughly 5”9 and 130 pounds ...
2,2gdooo,Rigor with antibiotics? Should I go to the hos...,I'm a sixteen year old female who has been pre...,,AskDocs,"{'a_id': ['cki7ezg'], 'text': ['This could be ...",[],[],[],Rigor with antibiotics? Should I go to the hos...,I'm a sixteen year old female who has been pre...
3,38xo1c,"Lost ~1.5L of blood, feeling very dizzy and we...","I'm a 19 year old male, 5'9"", and I had a sept...",,AskDocs,"{'a_id': ['cryoumi', 'crytvxa'], 'text': ['How...",[],[],[],"Lost ~1.5L of blood, feeling very dizzy and we...","I'm a 19 year old male, 5'9"", and I had a sept..."
4,2z1869,"My rash has stumped an NP, a PA, and three MDs","* 30\n* female\n* 5' 8""\n* 135 pre-pregnancy, ...",,AskDocs,"{'a_id': ['cpesedm', 'cpetqe4', 'cpf5d7o', 'cp...",[],[http://imgur.com/a/M5ZRj],[],"My rash has stumped an NP, a PA, and three MDs","* 30\n* female\n* 5' 8""\n* 135 pre-pregnancy, ..."


In [42]:

# Save 
sampled_train_df[["q_id","title_clean","selftext_clean"]].to_parquet("../data/raw/askdocs/askdocs_sample5.parquet", index=False)

Viewing Gemini results

In [9]:
result = pd.read_csv("../results/exp4_askdocs_gemini.csv", keep_default_na=False)

In [29]:
result.head()

,id,model,prediction,meta
0,9loutr,gemini,"Discovering a hard, disc-like area in your bre...","{""note"": ""real-world eval""}"
1,4fkkjd,gemini,"Yes, you should see a physician as soon as pos...","{""note"": ""real-world eval""}"
2,7m55bg,gemini,The oxygen gas itself does not expire or go ba...,"{""note"": ""real-world eval""}"
3,671fhy,gemini,I am incredibly sorry to hear about your broth...,"{""note"": ""real-world eval""}"
4,96n2tr,gemini,Based on the typical characteristics used in s...,"{""note"": ""real-world eval""}"


In [30]:
print(result["prediction"][0])

Discovering a hard, disc-like area in your breast can be caused by a number of conditions, and it is essential to have it evaluated by a healthcare professional. Benign (non-cancerous) possibilities include fibroadenomas, which are solid, rubbery lumps, cysts, which are fluid-filled sacs, or dense areas of tissue related to fibrocystic changes. However, a new, hard, or persistent lump can also be a sign of breast cancer. Because it is impossible to determine the cause without a physical examination and likely imaging tests like a mammogram or ultrasound, you should schedule an appointment with your doctor or a gynecologist as soon as possible. Prompt medical evaluation is the only way to get an accurate diagnosis, relieve your anxiety, and ensure you receive any necessary treatment.


# Test Dataset

In [3]:
# Load English test split
test_dataset = load_dataset("ju-resplande/askD", split="test_en")

print(test_dataset[0])

{'q_id': 'd0fwkq', 'title': 'How do I clean my husbands open wound from limestone crushing his arm??? 33 Male', 'selftext': "Race: Caucasian  \nHeight: 5'10  \nWeight: 200 pounds/90.72 Kilo  \nMedications: Clindamycin 300MG Capsules taken 3x a day and  Sulfameth/Trimethoprim 800/160MG TB taken 2x a day  \nThey also prescribed him oxycodone but he refused to fill the prescription as he doesn't want to become addicted to it.  \n\n\nOkay, so a little back story. Yesterday my husband was at work (he works at a monument company and cuts stone) and about 2000 pounds of limestone landed on his left forearm (He moved just in time so it didn't hit and crush his head).  Miraculously it didn't get broken (xrays were taken at hospital) but he has a nasty gash that's an open wound.  The hospital didn't tell him how to clean the area and I can visibly see blood seeping through the gauze and ace bandage as he's sleeping.  We have to go out to get stuff for his arm (gauze and things) and I don't know 

In [43]:
df = pd.DataFrame(test_dataset)

In [44]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,d0fwkq,How do I clean my husbands open wound from lim...,Race: Caucasian \nHeight: 5'10 \nWeight: 200...,,AskDocs,"{'a_id': ['ez9hg94', 'ez9qzj8'], 'text': ['If ...",[],[],[]
1,d0g2k1,My upper body is having some sort of reaction ...,My upper body feels like its enduring a consta...,,AskDocs,"{'a_id': ['ezbvtgo'], 'text': ['I am not a doc...",[],[],[]
2,d0hbbn,I'm afraid I have an undiagnosed mental or per...,I'm a 17 year old white male that is clean. I ...,,AskDocs,"{'a_id': ['ezaszir'], 'text': ['Possible Antis...",[],[],[]
3,d0i450,I may have accidentally inhaled hair dye for 7...,[deleted],,AskDocs,"{'a_id': ['ez9soz4'], 'text': ['You already co...",[],[],[]
4,d0j05h,After bursted Cyst and an operation my girlsfr...,"Girl\nAge:19\n56kg\nnonsmoker, otherwise very ...",,AskDocs,"{'a_id': ['ez9r4d5', 'eza3j35'], 'text': ['Not...",[],[],[https://www.womenshealth.gov/a-z-topics/ovari...


In [45]:
len(df)

5198

In [46]:
len(df["q_id"].unique())

5198

In [47]:
# --- Drop [deleted]/[removed]/empty ---
BAD = {"[deleted]", "[removed]"}
df = df[~df["selftext"].astype(str).str.strip().isin(BAD)]
df = df[~df["title"].astype(str).str.strip().isin(BAD)]
df = df[df["selftext"].astype(str).str.strip() != ""]


In [48]:
len(df)

3612

In [49]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls
0,d0fwkq,How do I clean my husbands open wound from lim...,Race: Caucasian \nHeight: 5'10 \nWeight: 200...,,AskDocs,"{'a_id': ['ez9hg94', 'ez9qzj8'], 'text': ['If ...",[],[],[]
1,d0g2k1,My upper body is having some sort of reaction ...,My upper body feels like its enduring a consta...,,AskDocs,"{'a_id': ['ezbvtgo'], 'text': ['I am not a doc...",[],[],[]
2,d0hbbn,I'm afraid I have an undiagnosed mental or per...,I'm a 17 year old white male that is clean. I ...,,AskDocs,"{'a_id': ['ezaszir'], 'text': ['Possible Antis...",[],[],[]
4,d0j05h,After bursted Cyst and an operation my girlsfr...,"Girl\nAge:19\n56kg\nnonsmoker, otherwise very ...",,AskDocs,"{'a_id': ['ez9r4d5', 'eza3j35'], 'text': ['Not...",[],[],[https://www.womenshealth.gov/a-z-topics/ovari...
6,d0kve9,Need advise regarding my Wife's medical case,"Respected Doctors ,\n\nWe are passing through ...",,AskDocs,"{'a_id': ['ezb4cib', 'ezaqwtr'], 'text': ['Nee...",[],[https://drive.google.com/drive/folders/1jxPWr...,[]


In [50]:
# --- Keep clean columns ---
df["title_clean"] = df["title"].map(clean_text)
df["selftext_clean"] = df["selftext"].map(clean_text)

In [51]:
df.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,title_clean,selftext_clean
0,d0fwkq,How do I clean my husbands open wound from lim...,Race: Caucasian \nHeight: 5'10 \nWeight: 200...,,AskDocs,"{'a_id': ['ez9hg94', 'ez9qzj8'], 'text': ['If ...",[],[],[],How do I clean my husbands open wound from lim...,Race: Caucasian \nHeight: 5'10 \nWeight: 200...
1,d0g2k1,My upper body is having some sort of reaction ...,My upper body feels like its enduring a consta...,,AskDocs,"{'a_id': ['ezbvtgo'], 'text': ['I am not a doc...",[],[],[],My upper body is having some sort of reaction ...,My upper body feels like its enduring a consta...
2,d0hbbn,I'm afraid I have an undiagnosed mental or per...,I'm a 17 year old white male that is clean. I ...,,AskDocs,"{'a_id': ['ezaszir'], 'text': ['Possible Antis...",[],[],[],I'm afraid I have an undiagnosed mental or per...,I'm a 17 year old white male that is clean. I ...
4,d0j05h,After bursted Cyst and an operation my girlsfr...,"Girl\nAge:19\n56kg\nnonsmoker, otherwise very ...",,AskDocs,"{'a_id': ['ez9r4d5', 'eza3j35'], 'text': ['Not...",[],[],[https://www.womenshealth.gov/a-z-topics/ovari...,After bursted Cyst and an operation my girlsfr...,"Girl\nAge:19\n56kg\nnonsmoker, otherwise very ..."
6,d0kve9,Need advise regarding my Wife's medical case,"Respected Doctors ,\n\nWe are passing through ...",,AskDocs,"{'a_id': ['ezb4cib', 'ezaqwtr'], 'text': ['Nee...",[],[https://drive.google.com/drive/folders/1jxPWr...,[],Need advise regarding my Wife's medical case,"Respected Doctors ,\nWe are passing through gr..."


In [52]:
# --- Sample 30 for experiment ---
sample = df.sample(n=30, random_state=42).reset_index(drop=True)

In [53]:
sample.head()

,q_id,title,selftext,document,subreddit,answers,title_urls,selftext_urls,answers_urls,title_clean,selftext_clean
0,djt2fh,Unusual (and life-destroying) gyno symptoms,"Hi there, I'm writing here as a last resort as...",,AskDocs,"{'a_id': ['f492b1u', 'f494c33', 'f49dyve', 'f4...",[],[],[https://www.groupbstrepinternational.org/what...,Unusual (and life-destroying) gyno symptoms,"Hi there, I'm writing here as a last resort as..."
1,695g7e,"Every time I eat food, I feel a dull ache in t...",Every time I eat any kind of food (though it d...,,AskDocs,"{'a_id': ['dh4156s', 'dh43cq5', 'dh41b9k'], 't...",[],[],[],"Every time I eat food, I feel a dull ache in t...",Every time I eat any kind of food (though it d...
2,d129bf,Extreme aggression outbursts. 30M,"I have been having aggression issues since, I ...",,AskDocs,"{'a_id': ['ezho53q', 'ezh642f', 'ezh5g09'], 't...",[],[],[],Extreme aggression outbursts. 30M,"I have been having aggression issues since, I ..."
3,eeyse3,Need advice about Hypertensive Crisis. Worried...,My mum( 60F ) was admitted to the emergency ro...,,AskDocs,"{'a_id': ['fbxseg6', 'fbxcg62'], 'text': ['Sup...",[],[],[],Need advice about Hypertensive Crisis. Worried...,My mum( 60F ) was admitted to the emergency ro...
4,d9ngm9,Can Marijuana/Cannabis Oil help with reflux?,"30M, 6ft, 187ibs, had gallbladder removed 4 ye...",,AskDocs,"{'a_id': ['f1k5a2t'], 'text': ['You'll be hard...",[],[],[],Can Marijuana/Cannabis Oil help with reflux?,"30M, 6ft, 187ibs, had gallbladder removed 4 ye..."


In [55]:
#save 
sample[["q_id","title_clean","selftext_clean"]].to_parquet("../data/processed/askdocs/askdocs_test_set.parquet")

In [56]:
test = pd.read_parquet("../data/processed/askdocs/askdocs_test_set.parquet")
test.head()

,q_id,title_clean,selftext_clean
0,djt2fh,Unusual (and life-destroying) gyno symptoms,"Hi there, I'm writing here as a last resort as..."
1,695g7e,"Every time I eat food, I feel a dull ache in t...",Every time I eat any kind of food (though it d...
2,d129bf,Extreme aggression outbursts. 30M,"I have been having aggression issues since, I ..."
3,eeyse3,Need advice about Hypertensive Crisis. Worried...,My mum( 60F ) was admitted to the emergency ro...
4,d9ngm9,Can Marijuana/Cannabis Oil help with reflux?,"30M, 6ft, 187ibs, had gallbladder removed 4 ye..."
